# 實際讀取檔

In [1]:
import tensorflow as tf
import mediapipe as mp
import numpy as np
import cv2

In [2]:
interpreter = tf.lite.Interpreter(model_path='model/pose017.tflite')
interpreter.allocate_tensors()

# 获取输入和输出张量的索引
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [10]:
def cos1(ax,ay,bx,by,cx,cy):#計算角度
    #handangle1=((cx-bx)*(ax-bx)+(cy-by)*(ay-by))/(((cx-bx)**2)**0.5+((cy-by)**2)**0.5)
    handangle1=((ax-bx)*(cx-bx)+(ay-by)*(cy-by))/((((ax-bx)**2+(ay-by)**2)**0.5)*(((cx-bx)**2+(cy-by)**2)**0.5))
    handangle2=np.degrees(np.arccos(handangle1))
    return handangle2
def howlong(ax,ay,bx,by):#計算兩點之間的長度
    fingerlong=round(abs(((ax-bx)**2+(ay-by)**2)**0.5),2)
    return fingerlong
cap=cv2.VideoCapture(0)
mpHands = mp.solutions.hands #手部追蹤模組
hands=mpHands.Hands()
mpDraw=mp.solutions.drawing_utils
num=[1,5,9,13,17]
THRESHOLD = 0.85

In [21]:
while True:
    ret,img=cap.read() # 布林值變數及影像變數
    imgH=img.shape[0]
    imgW=img.shape[1]
    if ret==True:
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #把圖片格式轉換成mediapipe想要的
        result=hands.process(imgRGB)
        # print(result.multi_hand_landmarks)
        if result.multi_hand_landmarks:
            for handlms in result.multi_hand_landmarks:#handlms=每一隻手
                mpDraw.draw_landmarks(img,handlms,mpHands.HAND_CONNECTIONS)
                hand=[]
                handangle=[]
                handlong=[]
                long0_5=0
                long0_17=0
                long5_17=0
                allcsvdata=[]
                Proportion0_5=[]
                Proportion0_17=[]
                Proportion5_17=[]
                for i,lm in enumerate(handlms.landmark):
                    Xpoint=round(lm.x*imgW,2)
                    Ypoint=round(lm.y*imgH,2)
                    #print(i,Xpoint,Ypoint)
                    hand.append((Xpoint,Ypoint))
                    #print(hand)                     
                #for i in range(0,21):
                #    print(hand[i][0],hand[i][1])
                for i in num:
                    # 角度
                    saveangle=cos1(hand[0][0],hand[0][1],hand[i][0],hand[i][1],hand[i+1][0],hand[i+1][1])
                    handangle.append(round(saveangle,2))
                    saveangle=cos1(hand[i][0],hand[i][1],hand[i+1][0],hand[i+1][1],hand[i+2][0],hand[i+2][1])
                    handangle.append(round(saveangle,2))
                    saveangle=cos1(hand[i+1][0],hand[i+1][1],hand[i+2][0],hand[i+2][1],hand[i+3][0],hand[i+3][1])
                    handangle.append(round(saveangle,2))
                    # 長度
                    fingerlong=howlong(hand[0][0],hand[i][0],hand[0][1],hand[i][1])
                    handlong.append(fingerlong)
                    fingerlong=howlong(hand[i][0],hand[i+1][0],hand[i][1],hand[i+1][1])
                    handlong.append(fingerlong)
                    fingerlong=howlong(hand[i+1][0],hand[i+2][0],hand[i+1][1],hand[i+2][1])
                    handlong.append(fingerlong)
                    fingerlong=howlong(hand[i+2][0],hand[i+3][0],hand[i+2][1],hand[i+3][1])
                    handlong.append(fingerlong)
                long0_17=fingerlong=howlong(hand[0][0],hand[17][0],hand[0][1],hand[17][1])
                long0_5=fingerlong=howlong(hand[0][0],hand[5][0],hand[0][1],hand[5][1])
                long5_17=fingerlong=howlong(hand[5][0],hand[17][0],hand[5][1],hand[17][1])
                for i in range(0,20):
                    savee=round(handlong[i]/long0_5,5)
                    Proportion0_5.append(savee)
                    savee=round(handlong[i]/long0_17,5)
                    Proportion0_17.append(savee)
                    savee=round(handlong[i]/long5_17,5)
                    Proportion5_17.append(savee)
                #print("handangle:",handangle)
                #print("handlong:",handlong)
                #print("Proportion0_5:",Proportion0_5)
                #print("Proportion0_17:",Proportion0_17)
                #print("Proportion0_17:",Proportion5_17)
                allcsvdata.extend(handangle)
                allcsvdata.extend(handlong)
                allcsvdata.extend(Proportion0_5)
                allcsvdata.extend(Proportion0_17)
                allcsvdata.extend(Proportion5_17)
                #print(allcsvdata)
                allcsvdata = np.array([allcsvdata], dtype=np.float32)
                interpreter.set_tensor(input_details[0]['index'], allcsvdata)
                interpreter.invoke()
                output_data = interpreter.get_tensor(output_details[0]['index'])
                predicted_classes = np.argmax(output_data, axis=1)
                if np.max(output_data) < THRESHOLD:
                    print("-1")
                else:
                    predicted_classes = np.argmax(output_data, axis=1)
                    #if predicted_classes< 4:
                    #    print("-1")
                    #else:
                    #print(output_data)
                    print(predicted_classes)
            
        cv2.imshow('img',img)# 顯示每一偵

    if cv2.waitKey(1) ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[6]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
[5]
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
[4]
-1
[4]
[4]
[4]
[4]
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
[4]
[4]
[4]
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
[4]
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
[5]
[5]
-1
-1
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[

In [11]:
import numpy as np

while True:
    ret, img = cap.read()
    imgH = img.shape[0]
    imgW = img.shape[1]
    if ret == True:
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(imgRGB)
        if result.multi_hand_landmarks:
            for handlms in result.multi_hand_landmarks:
                mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                allcsvdata = []  # 初始化为空列表
                for i, lm in enumerate(handlms.landmark):
                    Xpoint = lm.x
                    Ypoint = lm.y
                    allcsvdata.extend([Xpoint, Ypoint])  # 将Xpoint和Ypoint追加到allcsvdata列表中

                allcsvdata = np.array([allcsvdata], dtype=np.float32)  # 将列表转换为NumPy数组
                interpreter.set_tensor(input_details[0]['index'], allcsvdata)
                interpreter.invoke()
                output_data = interpreter.get_tensor(output_details[0]['index'])
                predicted_classes = np.argmax(output_data, axis=1)
                if np.max(output_data) < THRESHOLD:
                    print("-1")
                else:
                    predicted_classes = np.argmax(output_data, axis=1)
                    print(predicted_classes)
                    # 在螢幕左上角顯示動作編號
                    action_text = "Action: {}".format(predicted_classes)
                    cv2.putText(img, action_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


        cv2.imshow('img', img)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
-1
-1
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
-1
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
[2]
[2]
[2]
-1
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
-1
-1
-1
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
-1
-1
[2]
[2]
[2]
[2]
[2]
[2]
-1
[2]
[2]
[2]
[2]
[2]
-1
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
-1
-1
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
-1
[2]
[2]
[2]
-1
-1
-1
-1
-1
-1
-1
-1
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
-1
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]


# 模型正確測試檔

In [ ]:
import pandas as pd

# 讀取CSV檔案
data = pd.read_csv('pose.csv')

# 分離分類和資料
classes = data.iloc[:, 0]  # 第一列是分類
data = data.iloc[:, 1:]  # 從第二列開始是資料

# 轉換為numpy array
data_array = np.array(data, dtype=np.float32)

# 進行其他處理或預測
# ...

# 使用for迴圈進行判斷
for row in data_array:
    # 執行您的判斷
    # ...
    #print(row)
    row = np.array([row], dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], row)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_classes = np.argmax(output_data, axis=1)
    print(predicted_classes)


[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[2]
[0]
[2]
[0]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]


In [ ]:
import pandas as pd
import numpy as np

# 讀取CSV檔案
data = pd.read_csv('pose.csv')

# 分離分類和資料
classes = data.iloc[:, 0]  # 第一列是分類
data = data.iloc[:, 1:]  # 從第二列開始是資料

# 轉換為numpy array
data_array = np.array(data, dtype=np.float32)

# 使用for迴圈進行判斷
for i, row in enumerate(data_array):
    # 印出分類值
    class_value = classes[i]
    print("分類:", class_value)

    # 執行您的判斷
    # ...
    row = np.array([row], dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], row)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_classes = np.argmax(output_data, axis=1)
    print("預測分類:", predicted_classes)
    


分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [2]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [0]
分類: 11
預測分類: [

: 